<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2023/blob/main/Week9/Histogram_Strategy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wf8c1v3k
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wf8c1v3k
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=c4284df48c6f6f5a51d3c0e3d544960c6711f60e75946480332b4c823dedce10
  Stored in directory: /tmp/pip-ephem-wheel-cache-ovwdbmpo/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
%%cu
#include <stdio.h>
#include <cstdlib> //rand() function


inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

void histogram_CPU(int *data, const int N, int* histogram, const int Nbins)
{
	for(int i=0; i<N; ++i)
	{
		histogram[data[i]]++;		
	}
}

__global__ void histogram_GPU_1( int *data, const int N, int *histo) {
	int i = threadIdx.x + blockIdx.x * blockDim.x;    
	 int section_size = (N-1)/(blockDim.x *gridDim.x)+1; 
	int start = i*section_size; //All threads handle blockDim.x * gridDim.x consecutive elements 
	for (int k = 0; k < section_size; k++) 
	{
		if (start+k < N) 
		{ 
			atomicAdd(&(histo[data[start+k]]), 1); 
		} 
	}  
}


int main() 
{ 
	int *data;
	int *histogram;
	int *histogramGPU; 
	const int Nbins = 256;
	const int N = 1024 * 1024;
	const int threadsPerBlock = 256;

	#define imin(a,b) (a<b?a:b)

	const int blocksPerGrid =  imin( 32, (N+threadsPerBlock-1) / threadsPerBlock );
 
	// Allocate Unified Memory -- accessible from CPU or GPU
	checkCudaErr(cudaMallocManaged(&data, N*sizeof(int)), "cudaMallocManaged1 data");
	checkCudaErr(cudaMallocManaged(&histogram, Nbins*sizeof(int)), "cudaMallocManaged2 histogram"); 
	checkCudaErr(cudaMallocManaged(&histogramGPU, Nbins*sizeof(int)), "cudaMallocManaged2 histogramGPU"); 

	// fill in the data with random values between 0-255
	for (int i=0; i<N; i++) {
		data[i] = (rand() % Nbins); 
	} 

	// some events to count the execution time
  cudaEvent_t start, stop;
	float cpu_elapsed_time_ms, gpu_1_elapsed_time_ms; 

  cudaEventCreate(&start);
  cudaEventCreate(&stop);
	
	// start to count execution time of CPU version
	cudaEventRecord(start, 0);
 
  //calculate histogram on the CPU
	histogram_CPU(data, N, histogram, Nbins);

  // time counting terminate
  cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);

  //compute time elapsed on CPU 
  cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
  printf("Histogram(CPU) - Time: %f ms.\n", cpu_elapsed_time_ms);

 	//start to count execution time of GPU version
	cudaEventRecord(start, 0);

	//launch kernel
	histogram_GPU_1<<<blocksPerGrid,threadsPerBlock>>>( data, N, histogramGPU );

	// time counting terminate
  cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);

  // compute time elapse on GPU computing
  cudaEventElapsedTime(&gpu_1_elapsed_time_ms, start, stop);
  printf("Histogram_1(GPU) - Time: %f ms.\n", gpu_1_elapsed_time_ms);
	
	int allOK = 1;

	printf("+------------------+-------------------+\n");
  printf("| CPU Histogram[i] |  GPU Histogram[i] |\n");
	printf("+------------------+-------------------+\n");
  //compare results on CPU and GPU
	for(int i=0 ; i<Nbins; ++i)
 	{
		 printf("|      %5d       |      %5d        |\n", histogram[i], histogramGPU[i]);
      
		 if(histogram[i]!=histogramGPU[i])
		 {
		    printf("Histogram mismatch!!!");
				allOK=0;
				break;
		 }
	}
	printf("+------------------+-------------------+\n");
	
	if(allOK==1)
  	printf("Results correct on both CPU and GPU");

	// free memory on the gpu side
	checkCudaErr( cudaFree( data ) , "cudaFree1");
	checkCudaErr( cudaFree( histogram ) , "cudaFree2"); 
	checkCudaErr( cudaFree( histogramGPU ) , "cudaFree3"); 
	checkCudaErr( cudaDeviceReset(), "cudaDeviceReset");

	return 0;
}

Histogram(CPU) - Time: 4.835648 ms.
Histogram_1(GPU) - Time: 2.056224 ms.
+------------------+-------------------+
| CPU Histogram[i] |  GPU Histogram[i] |
+------------------+-------------------+
|       4083       |       4083        |
|       4074       |       4074        |
|       4073       |       4073        |
|       4107       |       4107        |
|       3910       |       3910        |
|       4082       |       4082        |
|       4173       |       4173        |
|       4074       |       4074        |
|       3957       |       3957        |
|       4073       |       4073        |
|       4072       |       4072        |
|       4077       |       4077        |
|       4098       |       4098        |
|       4105       |       4105        |
|       4112       |       4112        |
|       4070       |       4070        |
|       4152       |       4152        |
|       4061       |       4061        |
|       4101       |       4101        |
|       4105       |    